In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from scipy.sparse import csr_matrix
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
from fuzzywuzzy import process, fuzz
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pandas as pd

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
))

In [5]:
tracks_data = pd.read_csv('/Users/vatsalb/Desktop/Projects/Music Reccomender/tracks.csv')
features = ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
scaler = StandardScaler()
tracks_scaled = scaler.fit_transform(tracks_data[features])
tracks_sparse = csr_matrix(tracks_scaled)

In [6]:
tracks_data

,id,name,artists,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,['Uli'],6,126903,0,0.645,0.4450,0,-13.338,1,0.4510,0.6740,0.744000,0.1510,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,['Fernando Pessoa'],0,98200,0,0.695,0.2630,0,-22.136,1,0.9570,0.7970,0.000000,0.1480,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,['Ignacio Corsini'],0,181640,0,0.434,0.1770,1,-21.180,1,0.0512,0.9940,0.021800,0.2120,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,['Ignacio Corsini'],0,176907,0,0.321,0.0946,7,-27.961,1,0.0504,0.9950,0.918000,0.1040,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,['Dick Haymes'],0,163080,0,0.402,0.1580,3,-16.900,0,0.0390,0.9890,0.130000,0.3110,0.196,103.220,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416958,0w3Mfe4PIVjuFjJbe3OlYv,8am in Charlotte,Drake,84,266847,1,0.567,0.6820,4,-6.423,1,0.2740,0.6430,0.000000,0.5120,0.668,79.043,4
416959,2bTRpIxPniQmrSgwmtF1b2,Shake It Up,Trippie Redd,57,116680,1,0.774,0.6000,10,-6.728,0,0.1200,0.1210,0.000000,0.0924,0.389,156.073,4
416960,4kjI1gwQZRKNDkw1nI475M,MY EYES,Travis Scott,88,251249,1,0.455,0.6210,0,-7.694,0,0.0517,0.6510,0.000003,0.1230,0.280,119.043,4
416961,1PH2MDbcBAU094DlgTIND1,THANK GOD,Travis Scott,83,184888,1,0.657,0.5530,10,-7.080,0,0.0394,0.2540,0.000000,0.1290,0.576,134.925,4


In [9]:


def read_data(path,scaler_optn=0):
    features = ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    tracks_data = pd.read_csv(path)
    cols = ['id', 'name', 'artists'] + features
    tracks_data = tracks_data[cols]
    if scaler_optn == 1:
        scaler = StandardScaler()
    else:
        scaler = MinMaxScaler()

    tracks_scaled = scaler.fit_transform(tracks_data[features])
    tracks_sparse = csr_matrix(tracks_scaled)
        
    return tracks_data, tracks_sparse

threshold = 80  
def get_close_matches(df, song, artist, threshold):
    df['combined'] = df['name'] + ' ' + df['artists']    
    song_matches = process.extract(song, df['combined'], limit=None, scorer=fuzz.token_set_ratio)
    artist_matches = process.extract(artist, df['combined'], limit=None, scorer=fuzz.token_set_ratio)    
    close_song_matches = [df['combined'][index] for match, score, index in song_matches if score >= threshold]
    close_artist_matches = [df['combined'][index] for match, score, index in artist_matches if score >= threshold]    
    close_matches = set(close_song_matches).intersection(set(close_artist_matches))
    filtered_df = df[df['combined'].isin(close_matches)]
    df.drop('combined', axis = 1, inplace = True)
    return filtered_df.drop('combined', axis=1)

def get_song_from_spotify(song_name, artist_name=None):
    search_query = song_name if not artist_name else f"{song_name} artist:{artist_name}"
    results = sp.search(q=search_query, limit=1, type='track')
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        audio_features = sp.audio_features(track['id'])[0]
        song_details = {
            'id': track['id'],
            'name': track['name'],
            'popularity': track['popularity'],
            'duration_ms': track['duration_ms'],
            'explicit': int(track['explicit']),
            'artists': ', '.join([artist['name'] for artist in track['artists']]),  # Artists are returned as a list
            'danceability': audio_features['danceability'],
            'energy': audio_features['energy'],
            'key': audio_features['key'],
            'loudness': audio_features['loudness'],
            'mode': audio_features['mode'],
            'speechiness': audio_features['speechiness'],
            'acousticness': audio_features['acousticness'],
            'instrumentalness': audio_features['instrumentalness'],
            'liveness': audio_features['liveness'],
            'valence': audio_features['valence'],
            'tempo': audio_features['tempo'],
            'time_signature': audio_features['time_signature'],
        }
        return song_details
    else:
        return None 

def update_database_with_song(df, song_info, csv_file_path):
    new_entry = pd.DataFrame([song_info])
    updated_df = pd.concat([df, new_entry], ignore_index=True)
    updated_df.to_csv(csv_file_path, index=False)
    return read_data(csv_file_path, scaler_optn=0) 

def get_song_details(df, song_name, artist_name=None, csv_file_path=None):
    matched_df = get_close_matches(df, song_name, artist_name, threshold)
    if not matched_df.empty:
        print("Song found in the local database.")
        return matched_df
    else:
        print("Song not found in the local database. Searching on Spotify...")
        song_info = get_song_from_spotify(song_name, artist_name)
        if song_info:
            print("Song found on Spotify. Updating local database...")
            df, _ = update_database_with_song(df, song_info, csv_file_path) 
            print('Updated local database, fetching details...')
            matched_df = get_close_matches(df, song_name, artist_name, threshold)
            if not matched_df.empty:
                print("Fetching new song from local")
                return matched_df
            # return get_song_details(df, song_name, artist_name, csv_file_path)
            else:
                print('Shit is broken again') 
        else:
            print("Song not found on Spotify.")
            return None
               
def get_recommendations_cosine(song_name, artist, tracks_data, tracks_sparse, top_n=10,):
    song_details = get_song_details(tracks_data,song_name, artist, csv_file_path = '/Users/vatsalb/Desktop/Projects/Music Reccomender/tracks.csv')
    track_id = song_details.iloc[0]['id']
    idx = tracks_data.index[tracks_data['id'] == track_id].tolist()[0]
    track_vector = tracks_sparse[idx]
    sim_scores = cosine_similarity(track_vector, tracks_sparse).flatten()
    indices = sim_scores.argsort()[-top_n-1:-1][::-1]
    recommended_track_ids = tracks_data['id'].iloc[indices]
    recommended_tracks = tracks_data.loc[tracks_data['id'].isin(recommended_track_ids), ['id', 'name', 'artists']]
    return recommended_tracks


In [8]:
csv_file_path = '/Users/vatsalb/Desktop/Projects/Music Reccomender/tracks.csv'
tracks_data, tracks_sparse = read_data(csv_file_path, scaler_optn=0)
recommended_tracks = get_recommendations_cosine('blood on the leaves','kanye west', tracks_data,tracks_sparse, top_n=10)
recommended_tracks

Song found in the local database.


,id,name,artists
73479,1boHm9rjmvzxpJASd9dnGK,Slow Jamz (feat. Kanye West & Jamie Foxx),"['Twista', 'Kanye West', 'Jamie Foxx']"
79310,0V4l4GQhgnWQGtCWpvA7va,Crew Love,"['Drake', 'The Weeknd']"
83566,4FxkQzIhsLAkuiqX5puvU6,Till It's Gone,['Yelawolf']
84318,1wsZ0cp5He4Wm4yohNjwIs,idfc - Tarro Remix,"['blackbear', 'Tarro']"
84718,1o0kWPueYo94LIjPYOE5Nf,Saint Pablo,['Kanye West']
107403,2jdw2tc29bqJwToyGvKgJm,Ninety,['Jaden']
235986,3SzBLvShr2FKp1qSaqK6cm,Mala Vida,['Nicki Nicole']
270880,6QdnKD1zwEgyOWtkrdzlOF,Hurt You,"['The Weeknd', 'Gesaffelstein']"
353296,4GHHEoXXWRt3Hnd9CDRycr,Тело,['LSP']
353864,0gQ11rTd3XI9sMoyv3TImt,Bismarck,"['MiyaGi', 'TumaniYO', 'KADI']"


In [75]:
recommended_tracks

,id,name,artists
83129,27NV2KxoQ8WuLMqlTDI61F,Often - Kygo Remix,"['The Weeknd', 'Kygo']"
85249,2u7mxWSeoqTXndK5e08jMp,Nowadays (feat. Landon Cube),"['Lil Skies', 'Landon Cube']"
85865,7vgTNTaEz3CsBZ1N4YQalM,Ghost Town,"['Kanye West', 'PARTYNEXTDOOR']"
86383,7rC5Pl8rQSX4myONQHYPBK,Mob Ties,['Drake']
87920,0DYNL4xbYWgNzppUamtyR4,"F*CK YOU, GOODBYE (feat. Machine Gun Kelly)","['The Kid LAROI', 'Machine Gun Kelly']"
88120,30KctD1WsHKTIYczXjip5a,Tequila Shots,['Kid Cudi']
276929,0NmuYnjETG3u3qx0OmEJev,Lil Bit,"['Nelly', 'Florida Georgia Line']"
276975,3k0YJnqMKRZb8swo86vCkq,drunk face,['Machine Gun Kelly']
276983,7ge7BHazYbVKi8qyZUX1Bm,Emotionally Scarred,['Lil Baby']
526615,5TRPicyLGbAF2LGBFbHGvO,Flashing Lights,"[['Kanye West', 'Dwele']]"
